# Финальный проект. Неделя 3

In [11]:
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords as nltk_stop_words
import pandas as pd
import numpy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
import pandas as pd

чтение тренировочного и тестового датасета

In [25]:
train_data = pd.read_csv('products_sentiment_train.tsv', sep = '\t', header = None)
train_data.columns = ('text','label')
test_data = pd.read_csv('products_sentiment_test.tsv', sep = '\t')


In [27]:
train_data.head()

,text,label
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


Проведем несколько экспериментов, попробовав несколько классификаторов и разные способы построения словарей.

Создадим словари с различными параметрами, которые будем перебирать для настройки модели в экспериментах.

In [110]:
vect_parameters = {'vect__ngram_range':((1,1), (1,2), (1,3)),
              'vect__stop_words':('english',nltk_stop_words.words('english'), None),
              'vect__min_df':(0, 1),
              'vect__max_df':(0.7, 0.8, 0.9)             
             }
logreg_parameters = {'logreg__penalty': ('l1', 'l2'),
                     'logreg__C': (0.001, 0.2, 0.8, 1.0),
                     
    
}
linearSVC_parameters = {
                        'linearSVC__loss':('hinge','squared_hinge'),
                        'linearSVC__C':(0.0001,0.3, 0.5,0.7,1.0)
                        
    
}
SGD_parameters = {'sgd__loss':('hinge','squared_hinge'),
    #'sgd__loss':('hinge','log', 'modified_huber', 'squared_hinge', 'perceptron'),
                  'sgd__penalty':('l1', 'l2'),
                  'sgd__epsilon':(0.1,0.01),
                  'sgd__n_iter':(5,15, 20)
    
}
tfidf_parameters = {#'tfidf__stop_words':('english',nltk_stop_words.words('english'), None),
                    #'tfidf__ngram_range':((1,1), (1,2), (1,3), (3,5)),
                    'tfidf__ngram_range':((1,3), (1,4), (2,4)),
                    #'tfidf__min_df':(1, 5,8,10),
                    'tfidf__max_df':(0.7, 0.8, 0.9),
                    'tfidf__norm':('l1','l2')
    
}


In [111]:
#функция которая возвращает лучшую модель, после перебора по сетке
def experiment(name, pipeline, dict_params, model_params):
    print name, ':'
    params = dict_params.copy()
    params.update(model_params)
    optimizer = GridSearchCV(pipeline,params, scoring='accuracy')
    optimizer.fit(train_data['text'], train_data['label'])
    print optimizer.best_score_
    print optimizer.best_params_
    print optimizer
    return optimizer.best_estimator_
    

    1. CountVectorizer + LogisticRegression

In [7]:
pipeline = Pipeline(steps=[('vect', CountVectorizer()), ('logreg', LogisticRegression())])

In [50]:
%%time
estimator1 = experiment('CountVectorizer + LogisticRegression', pipeline, vect_parameters, logreg_parameters)

CountVectorizer + LogisticRegression :
0.774
{'vect__ngram_range': (1, 1), 'vect__max_df': 0.7, 'logreg__penalty': 'l2', 'logreg__C': 1.0, 'vect__min_df': 0, 'vect__stop_words': None}


    2. CountVectorizer + LinearSVC

In [58]:
pipeline1 = Pipeline(steps=[('vect', CountVectorizer()), ('linearSVC', LinearSVC())])

In [65]:
%%time
estimator2 = experiment('CountVectorizer + LinearSVC', pipeline1, vect_parameters, linearSVC_parameters)

CountVectorizer + LinearSVC :
0.765
{'linearSVC__C': 0.3, 'vect__ngram_range': (1, 1), 'vect__max_df': 0.7, 'vect__min_df': 0, 'vect__stop_words': None, 'linearSVC__loss': 'squared_hinge'}
Wall time: 3min 40s


    3. CountVectorizer +SGDClassifier

In [66]:
pipeline2 = Pipeline(steps=[('vect', CountVectorizer()), ('sgd', SGDClassifier())])

In [70]:
%%time 
estimator3 = experiment('CountVectorizer +SGDClassifier', pipeline2, vect_parameters, SGD_parameters)

CountVectorizer +SGDClassifier :
0.779
{'vect__ngram_range': (1, 3), 'vect__max_df': 0.8, 'sgd__loss': 'log', 'sgd__penalty': 'l2', 'sgd__epsilon': 0.1, 'vect__min_df': 1, 'vect__stop_words': None, 'sgd__n_iter': 10}
Wall time: 38min 53s


    4. TFIdfVectorizer + LogisticRegression

In [78]:
pipeline3 = Pipeline(steps=[('tfidf', TfidfVectorizer()), ('logreg', LogisticRegression())])

In [90]:
%%time 
estimator4 = experiment('TFIdfVectorizer +LogisticRegression', pipeline3, tfidf_parameters, logreg_parameters)

TFIdfVectorizer +LogisticRegression :
0.7575
{'tfidf__ngram_range': (1, 1), 'logreg__penalty': 'l2', 'tfidf__stop_words': None, 'tfidf__max_df': 0.7, 'logreg__C': 1.0, 'tfidf__norm': 'l2'}
Wall time: 4min 44s


    5. TFIdfVectorizer + LinearSVC

In [95]:
pipeline4 = Pipeline(steps=[('tfidf', TfidfVectorizer()),('linearSVC', LinearSVC())])

In [96]:
%%time
estimator5 = experiment('TFIdfVectorizer+LinearSVC', pipeline4, tfidf_parameters, linearSVC_parameters)

TFIdfVectorizer+LinearSVC :
0.7845
{'linearSVC__C': 1.0, 'tfidf__ngram_range': (1, 2), 'tfidf__stop_words': None, 'tfidf__max_df': 0.7, 'tfidf__norm': 'l2', 'linearSVC__loss': 'hinge'}
Wall time: 5min 39s


    6. TFIdfVectorizer + SGDClassifier

In [97]:
pipeline5 = Pipeline(steps=[('tfidf', TfidfVectorizer()),('sgd', SGDClassifier())])

In [112]:
%%time
estimator6 = experiment('TFIdfVectorizer+SGDClassifier', pipeline5, tfidf_parameters, SGD_parameters)

TFIdfVectorizer+SGDClassifier :
0.791
{'tfidf__ngram_range': (1, 3), 'sgd__loss': 'hinge', 'sgd__penalty': 'l2', 'tfidf__max_df': 0.8, 'sgd__epsilon': 0.01, 'tfidf__norm': 'l2', 'sgd__n_iter': 15}
GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'tfidf__max_df': (0.7, 0.8, 0.9), 'sgd__epsilon': (0.1, 0.01), 'tfidf__norm': ('l1', 'l2'), 'tfidf__ngram_range': ((1, 3), (1, 4), (2, 4)), 'sgd__loss': ('hinge', 'squared_hinge'), 'sgd__penalty': ('l1', 'l2'), 'sgd__n_iter': (5, 15, 20)},
       pre_dispatch='2*n_jo

Вывод: была проведена серия экспериментов, по итогу которых лучшей оказалась модель TF-IDF + SGDClassifier. Эта модель и была выбрана для отправки результатов предсказаний на Kaggle.

In [154]:
estimator6.fit(train_data['text'], train_data['label'])
pred = estimator6.predict(test_data['text'])
predictions_df = pd.DataFrame(data=pred, columns=['y'])
predictions_df.to_csv('predictions.csv', index_label='Id')
